In [ ]:
#本记事本就是用来测试模型使得最后出来一个表格————每一个的bisai_id都有一个真是的real_cold情况和predict_cold情况

In [1]:
#用来测试textRNN_half2在非平衡数据集中的准确率
#shuffle=False，因为后续计算需要
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import sys
import pandas as pd
import numpy as np
import csv
import random
import re
from sklearn.decomposition import TruncatedSVD
from torch.nn.utils.rnn import pad_sequence#用来填充序列
import time
from prefetch_generator import BackgroundGenerator
from torch.utils.tensorboard import SummaryWriter
from pywick.optimizers.nadam import Nadam#使用pywick包里的nadam优化器
from torch.optim import lr_scheduler
with open('/home/jsy/data/cidlist_complete.csv') as f:
    reader = csv.reader(f)
    cidlist = [row[1] for row in reader]#得到cid对应表
cidlist = list(map(float,cidlist))#把各个元素字符串类型转成浮点数类型
class DataLoaderX(DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())
class BisaiDataset(Dataset):#数据预处理器
    def __init__(self,filepath):#filepath是个列表
        self.lablelist = pd.read_csv('/home/jsy/data/lablelist.csv',index_col = 0)#比赛id及其对应赛果的列表
        self.filelist0 = list()
        for x in filepath:#把filepath这个列表中的所有目录下的路径组合成一个列表
            self.filelist0+=[i+'/'+k for i,j,k in os.walk(x) for k in k]#得到所有csv文件的路径列表
        #self.filelist0 = [i+'/'+k for i,j,k in os.walk(filepath) for k in k]#得到所有csv文件的路径列表
        self.filelist = [data_path for data_path in self.filelist0 if int(re.findall(r'/(\d*?).csv',data_path)[0]) in  self.lablelist.index]#只保留有赛果的文件路径
        self.lables = {'win':0,'draw':1,'lose':2}#分类问题要从0开始编号,而且要对应好了表中的顺序编，
    
    def __getitem__(self, index):
        #todo
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        #这里需要注意的是，第一步：read one data，是一个dat
        data_path = self.filelist[index]
        bisai_id = int(re.findall(r'/(\d*?).csv',data_path)[0])
        # 2. Preprocess the data (e.g. torchvision.Transform).
        data = self.csv2frame(data_path)
        # 3. Return a data pair (e.g. image and label).
        lable = self.lablelist.loc[bisai_id].result
        lable = self.lables[lable]
        return data,lable,bisai_id   
       
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return len(self.filelist)

    def csv2frame(self,filepath):#给出单场比赛的csv文件路径，并转化成帧列表和对应变帧时间列表，以及比赛结果
        data = pd.read_csv(filepath)#读取文件
        data = data.drop(columns=['league','zhudui','kedui','companyname'])#去除非数字的列
        frametimelist=data.frametime.value_counts().sort_index(ascending=False).index#将frametime的值读取成列表
        framelist =list()#framelist为一个空列表,长度与frametimelist相同,一定要规定好具体形状和float类型，否则dataloader无法读取
        '''
        此处两个循环算法太慢，用pandas更慢，完全抛弃pandas后，数据处理速度从109秒降到了10秒，降到10秒后cpu利用率20%，再往上提也提不上去了，可能需要C++或C来写了
        '''
        new_data = np.array(data)
        lables = new_data[:,0]
        if len(frametimelist)>250:
            frametimelist = [frametimelist[0]]+random.sample(list(frametimelist)[1:-1],248)+[frametimelist[-1]]#如果长度大于500,保留头尾，并在中间随机抽取498个，共计500个
            frametimelist.sort(reverse=True)#并降序排列
        for i in frametimelist:
            state = new_data[lables==i]#从第一次变盘开始得到当次转移
            #state = np.array(state)#不必转成numpy多维数组，因为已经是了
            state = np.delete(state,(0,1), axis=-1)#去掉frametime和cid
            #在填充成矩阵之前需要知道所有数据中到底有多少个cid
            framelist.append(state)
        frametimelist = np.array(frametimelist)
        vectensor = self.mrx2vec(framelist)
        len_frame = vectensor.shape[0]
        if len_frame<250:
            vectensor = np.concatenate((np.zeros((250-len_frame,10),dtype=np.float64),vectensor),axis=0)#如果不足500，则在前面用0填充
        vectensor = torch.from_numpy(vectensor)
        return vectensor#传出一个帧列表,也可以把frametimelist一并传出来，此处暂不考虑位置参数的问题
    
    def tsvd(self,frame):
        tsvd = TruncatedSVD(1)
        if frame.shape[0] != 1:
            newframe = tsvd.fit_transform(np.transpose(frame))#降维成（1,10）的矩阵
        else:
            return frame.reshape((10,1))#第一行需要reshape一下
        return newframe
    
    def mrx2vec(self,flist):#把截断奇异值的方法把矩阵变成向量(matrix2vec/img2vec)，传入：len(frametimelist)*(306*10),传出：len(frametimelist)*10
        vectensor = np.array(list(map(self.tsvd,flist))).squeeze(2)
        #veclist = veclist.transpose()
        #vectensor = torch.from_numpy(veclist)#转成张量
        return vectensor#传出一个形状为(1,序列长度,10)的张量，因为后面传入模型之前，还需要做一下pad_sequence(0维是batch_size维)

 


class Lstm(nn.Module):#在模型建立之处就把它默认初始化
    def __init__(self):
        super().__init__()
        self.encoder = nn.LSTM(input_size=10, 
                                hidden_size=250,#选择对帧进行保留首尾的均匀截断采样
                                num_layers=1,#暂时就只有一层
                                bidirectional=True)
        nn.init.orthogonal_(self.encoder.weight_ih_l0)
        nn.init.orthogonal_(self.encoder.weight_hh_l0)
        nn.init.constant_(self.encoder.bias_ih_l0,0.0)
        nn.init.constant_(self.encoder.bias_hh_l0,0.0)
        self.decoder = nn.Sequential(
            nn.Linear(1000, 250),#把LSTM的输出
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(250, 3)
        )
        nn.init.normal_(self.decoder[0].weight,mean=0.0)
        nn.init.constant_(self.decoder[0].bias, 0.0)
        nn.init.normal_(self.decoder[3].weight,mean=0.0)
        nn.init.constant_(self.decoder[3].bias, 0.0)

    def forward(self,inputs):
        output, _= self.encoder(inputs.permute(1,0,2))#inputs需要转置一下再输入lstm层，因为pytorch要求第0维为长度，第二维才是batch_size
        encoding = torch.cat((output[0], output[-1]), -1)#双向的lstm，就把两个都放进去
        return self.decoder(encoding)#把最后一个时间步的输出输入MLP


def get_parameter_number(model):#参数统计
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}



if __name__ == "__main__":
    checkpoint_path = '/home/jsy/log/checkpoints/checkpoint.pth'#ckpoint文件夹需要提前建立
    test_path1 = ['/home/jsy/train_20160701-20170531','/media/jsy/Samsung/train_20180211-20190224','/media/jsy/WD/train_20170601-20180210']
    test_path2 = ['/media/jsy/Samsung/test2','/home/jsy/test4']
    test_path = test_path1+test_path2
    test_set = BisaiDataset(test_path)#验证集
    print('数据集读取完成')
    test_loader = DataLoaderX(test_set, 64 ,shuffle=False,num_workers=4,pin_memory=True)#num_workers>0情况下无法在交互模式下运行
    print('dataloader准备完成')
    net = Lstm().double().cuda()#双精度
    print('网络构建完成')
    stat1 = get_parameter_number(net)
    print(str(stat1))
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        net.load_state_dict(checkpoint['model_state_dict'])
    #loss = nn.CrossEntropyLoss()
    print('开始验证......')
    net.eval()
    torch.cuda.empty_cache()#释放一下显存
    with torch.no_grad():#这样在验证时显存才不会爆
        test_output = torch.zeros((1,3))
        test_y = torch.zeros((1)).long()#得是long类型
        bisai_id_list = torch.zeros((1)).long()
        test_counter = 0
        #先测试test_loader1
        start = time.time()
        for x,y,z in iter(test_loader):
            x = x.double().cuda()
            output = net(x).cpu()#把输出转到内存
            test_output = torch.cat((test_output,output),0)#把这一个batch的输出连起来
            test_y = torch.cat((test_y,y),0)#把这一个batch的lable连起来
            bisai_id_list = torch.cat((bisai_id_list,z),0)#保存比赛预测之对应的id，供之后分析
            #torch.cuda.empty_cache()
            test_counter+=1
            print('test_set1已完成'+str(test_counter)+'个batch')
        #验证输出和验证lable的第一个元素都是0，鉴于到时候占比很小就不删除了
        print('计算结果......')
        #l_test = loss(test_output,test_y)#用整个验证集的输出和lable算一个总平均loss(nn.CrossEntropyLoss默认就是求平均值)
        prediction = torch.argmax(test_output, 1)#找出每场比赛预测输出的最大值的坐标
        correct = (prediction == test_y).sum().float()#找出预测正确的总个数
        accuracy = correct/len(test_y)#计算Top-1正确率,总共就三分类，就不看top-2的了
        end = time.time()
        print('test_set1测试完成,用时'+str(end-start)+'秒,准确率为'+str(accuracy))
    print('test_set的Top-1精确度为'+str(accuracy))
#下面将预测结果和对应比赛制成predict_lablelist
prediction = prediction.tolist()#把tensor转成list
bisai_id_list = bisai_id_list.tolist()
predict_lablelist = pd.DataFrame(data={'bisai_id':bisai_id_list,'prediction':prediction})
predict_lablelist.set_index(['bisai_id'], inplace=True)
predict_lablelist = predict_lablelist.drop(index=0)
predict_lablelist = predict_lablelist[~predict_lablelist.index.duplicated(keep='first')]#去掉重复index，否则会出错
    


数据集读取完成
dataloader准备完成
网络构建完成
{'Total': 775003, 'Trainable': 775003}
开始验证......
test_set1已完成1个batch
test_set1已完成2个batch
test_set1已完成3个batch
test_set1已完成4个batch
test_set1已完成5个batch
test_set1已完成6个batch
test_set1已完成7个batch
test_set1已完成8个batch
test_set1已完成9个batch
test_set1已完成10个batch
test_set1已完成11个batch
test_set1已完成12个batch
test_set1已完成13个batch
test_set1已完成14个batch
test_set1已完成15个batch
test_set1已完成16个batch
test_set1已完成17个batch
test_set1已完成18个batch
test_set1已完成19个batch
test_set1已完成20个batch
test_set1已完成21个batch
test_set1已完成22个batch
test_set1已完成23个batch
test_set1已完成24个batch
test_set1已完成25个batch
test_set1已完成26个batch
test_set1已完成27个batch
test_set1已完成28个batch
test_set1已完成29个batch
test_set1已完成30个batch
test_set1已完成31个batch
test_set1已完成32个batch
test_set1已完成33个batch
test_set1已完成34个batch
test_set1已完成35个batch
test_set1已完成36个batch
test_set1已完成37个batch
test_set1已完成38个batch
test_set1已完成39个batch
test_set1已完成40个batch
test_set1已完成41个batch
test_set1已完成42个batch
test_set1已完成43个batch
test_set1已完成44个batch
test_

test_set1已完成376个batch
test_set1已完成377个batch
test_set1已完成378个batch
test_set1已完成379个batch
test_set1已完成380个batch
test_set1已完成381个batch
test_set1已完成382个batch
test_set1已完成383个batch
test_set1已完成384个batch
test_set1已完成385个batch
test_set1已完成386个batch
test_set1已完成387个batch
test_set1已完成388个batch
test_set1已完成389个batch
test_set1已完成390个batch
test_set1已完成391个batch
test_set1已完成392个batch
test_set1已完成393个batch
test_set1已完成394个batch
test_set1已完成395个batch
test_set1已完成396个batch
test_set1已完成397个batch
test_set1已完成398个batch
test_set1已完成399个batch
test_set1已完成400个batch
test_set1已完成401个batch
test_set1已完成402个batch
test_set1已完成403个batch
test_set1已完成404个batch
test_set1已完成405个batch
test_set1已完成406个batch
test_set1已完成407个batch
test_set1已完成408个batch
test_set1已完成409个batch
test_set1已完成410个batch
test_set1已完成411个batch
test_set1已完成412个batch
test_set1已完成413个batch
test_set1已完成414个batch
test_set1已完成415个batch
test_set1已完成416个batch
test_set1已完成417个batch
test_set1已完成418个batch
test_set1已完成419个batch
test_set1已完成420个batch
test_set1已

test_set1已完成749个batch
test_set1已完成750个batch
test_set1已完成751个batch
test_set1已完成752个batch
test_set1已完成753个batch
test_set1已完成754个batch
test_set1已完成755个batch
test_set1已完成756个batch
test_set1已完成757个batch
test_set1已完成758个batch
test_set1已完成759个batch
test_set1已完成760个batch
test_set1已完成761个batch
test_set1已完成762个batch
test_set1已完成763个batch
test_set1已完成764个batch
test_set1已完成765个batch
test_set1已完成766个batch
test_set1已完成767个batch
test_set1已完成768个batch
test_set1已完成769个batch
test_set1已完成770个batch
test_set1已完成771个batch
test_set1已完成772个batch
test_set1已完成773个batch
test_set1已完成774个batch
test_set1已完成775个batch
test_set1已完成776个batch
test_set1已完成777个batch
test_set1已完成778个batch
test_set1已完成779个batch
test_set1已完成780个batch
test_set1已完成781个batch
test_set1已完成782个batch
test_set1已完成783个batch
test_set1已完成784个batch
test_set1已完成785个batch
test_set1已完成786个batch
test_set1已完成787个batch
test_set1已完成788个batch
test_set1已完成789个batch
test_set1已完成790个batch
test_set1已完成791个batch
test_set1已完成792个batch
test_set1已完成793个batch
test_set1已

test_set1已完成1117个batch
test_set1已完成1118个batch
test_set1已完成1119个batch
test_set1已完成1120个batch
test_set1已完成1121个batch
test_set1已完成1122个batch
test_set1已完成1123个batch
test_set1已完成1124个batch
test_set1已完成1125个batch
test_set1已完成1126个batch
test_set1已完成1127个batch
test_set1已完成1128个batch
test_set1已完成1129个batch
test_set1已完成1130个batch
test_set1已完成1131个batch
test_set1已完成1132个batch
test_set1已完成1133个batch
test_set1已完成1134个batch
计算结果......
test_set1测试完成,用时16356.431728363037秒,准确率为tensor(0.5235)
test_set的Top-1精确度为tensor(0.5235)


In [2]:
#把预测的lable和bidai_id对应起来，得到predict_bisai_id
#先得到prediction1对应的bisai_id
test_path1 = ['/home/jsy/train_20160701-20170531','/media/jsy/Samsung/train_20180211-20190224','/media/jsy/WD/train_20170601-20180210']
test_path2 = ['/media/jsy/Samsung/test2','/home/jsy/test4']
test_path = test_path1+test_path2
lablelist = pd.read_csv('/home/jsy/data/lablelist.csv',index_col = 0)#比赛id及其对应赛果的列表
filelist0 = list()
for x in test_path:#把filepath这个列表中的所有目录下的路径组合成一个列表
    filelist0+=[i+'/'+k for i,j,k in os.walk(x) for k in k]#得到所有csv文件的路径列表
#self.filelist0 = [i+'/'+k for i,j,k in os.walk(filepath) for k in k]#得到所有csv文件的路径列表
filelist = [data_path for data_path in filelist0 if int(re.findall(r'/(\d*?).csv',data_path)[0]) in  lablelist.index]#只保留有赛果的文件
bisai_id = [int(re.findall(r'/(\d*?).csv',data_path)[0]) for data_path in filelist]
lables = {'win':0,'draw':1,'lose':2}


In [3]:
#得到实际的冷门情况列表
coldlist =[] 
def classify(data_path):
    with open(data_path, 'rb') as f:  # 打开文件

    # 在文本文件中，没有使用b模式选项打开的文件，只允许从文件头开始,只能seek(offset,0)

        offset = -50  # 设置偏移量

        while True:

            """

            file.seek(off, whence=0)：从文件中移动off个操作标记（文件指针），正往结束方向移动，负往开始方向移动。

            如果设定了whence参数，就以whence设定的起始位为准，0代表从头开始，1代表当前位置，2代表文件最末尾位置。  

            """

            f.seek(offset, 2)  # seek(offset, 2)表示文件指针：从文件末尾(2)开始向前50个字符(-50)

            lines = f.readlines()  # 读取文件指针范围内所有行

            if len(lines) >= 2:  # 判断是否最后至少有两行，这样保证了最后一行是完整的

                last_line = lines[-1]  # 取最后一行

                break

            # 如果off为50时得到的readlines只有一行内容，那么不能保证最后一行是完整的

            # 所以off翻倍重新运行，直到readlines不止一行

            offset *= 2
    last_line_list=last_line.decode().split(',')#把字符串转成列表，分隔符为逗号
    peilv =[float(last_line_list[7]),float(last_line_list[8]),float(last_line_list[9])]#把三种赔率装起来
    bisai_id = int(re.findall(r'/(\d*?).csv',data_path)[0])
    lable =lablelist.loc[bisai_id].result
    lable = lables[lable]#找到对应场比赛的结果
    result = peilv[lable]
    if result == min(peilv):#如果预测赢了的那个是赔率最小的那个
        coldlist.append((bisai_id,'normal'))
    elif result == max(peilv):
        coldlist.append((bisai_id,'big_cold'))
    else:
        coldlist.append((bisai_id,'little_cold'))


        
        
list(map(classify,filelist))
real_cold_df = pd.DataFrame(data=coldlist,columns=['bisai_id','cold'])
real_cold_df.set_index(['bisai_id'], inplace=True)

In [4]:
#得到预测的冷门程度列表
predict_lablelist = predict_lablelist[~predict_lablelist.index.duplicated(keep='first')]#去掉重复index，否则会出错
predict_coldlist =[] 
def classify_predict(data_path):
    with open(data_path, 'rb') as f:  # 打开文件

    # 在文本文件中，没有使用b模式选项打开的文件，只允许从文件头开始,只能seek(offset,0)

        offset = -50  # 设置偏移量

        while True:

            """

            file.seek(off, whence=0)：从文件中移动off个操作标记（文件指针），正往结束方向移动，负往开始方向移动。

            如果设定了whence参数，就以whence设定的起始位为准，0代表从头开始，1代表当前位置，2代表文件最末尾位置。  

            """

            f.seek(offset, 2)  # seek(offset, 2)表示文件指针：从文件末尾(2)开始向前50个字符(-50)

            lines = f.readlines()  # 读取文件指针范围内所有行

            if len(lines) >= 2:  # 判断是否最后至少有两行，这样保证了最后一行是完整的

                last_line = lines[-1]  # 取最后一行

                break

            # 如果off为50时得到的readlines只有一行内容，那么不能保证最后一行是完整的

            # 所以off翻倍重新运行，直到readlines不止一行

            offset *= 2
    last_line_list=last_line.decode().split(',')#把字符串转成列表，分隔符为逗号
    peilv =[float(last_line_list[7]),float(last_line_list[8]),float(last_line_list[9])]#把三种赔率装起来
    bisai_id = int(re.findall(r'/(\d*?).csv',data_path)[0])
    lable = predict_lablelist.loc[bisai_id].prediction#此时的lable就是0,1,2的形式，就不用通过字典lables来转换了
    #lable = lables[lable]#找到对应场比赛的结果
    result = peilv[lable]
    if result == min(peilv):#如果预测赢了的那个是赔率最小的那个
        predict_coldlist.append((bisai_id,'normal'))
    elif result == max(peilv):
        predict_coldlist.append((bisai_id,'big_cold'))
    else:
        predict_coldlist.append((bisai_id,'little_cold'))


        
        
list(map(classify_predict,filelist))
predict_cold_df = pd.DataFrame(data=predict_coldlist,columns=['bisai_id','predict_cold'])
predict_cold_df.set_index(['bisai_id'], inplace=True)


In [5]:
a = real_cold_df.join(predict_cold_df)

In [6]:
a

,cold,predict_cold
bisai_id,,
741109,big_cold,normal
783547,little_cold,normal
783548,normal,normal
783550,normal,normal
783552,little_cold,normal
...,...,...
1053484,little_cold,normal
1053494,normal,normal
1053495,big_cold,normal


In [7]:
#在预测中为暴大冷，实际上也暴大冷的概率是0.4025
a.loc[(a['predict_cold']=='big_cold')&(a['cold']=='big_cold')].shape[0]/a.loc[(a['predict_cold']=='big_cold')].shape[0]

0.4025157232704403

In [8]:
#在预测中为暴小冷，实际上也暴小冷的概率是0.3641
a.loc[(a['predict_cold']=='little_cold')&(a['cold']=='little_cold')].shape[0]/a.loc[(a['predict_cold']=='little_cold')].shape[0]

0.36409480584972265

In [9]:
#在预测中为正常，实际上也正常的概率是0.5337
a.loc[(a['predict_cold']=='normal')&(a['cold']=='normal')].shape[0]/a.loc[(a['predict_cold']=='normal')].shape[0]

0.5337291722893791

In [10]:
#预测中为爆大冷，实际上为爆大冷或爆小冷的概率为0.5912
a.loc[(a['predict_cold']=='big_cold')&((a['cold']=='big_cold')|(a['cold']=='little_cold'))].shape[0]/a.loc[(a['predict_cold']=='big_cold')].shape[0]

0.5911949685534591

In [11]:
#预测中为爆冷（大冷或小冷），实际上也爆冷（大冷或小冷）的概率为0.6541
a.loc[(a['predict_cold']!='normal')&(a['cold']!='normal')].shape[0]/a.loc[(a['predict_cold']!='normal')].shape[0]

0.65406162464986

In [12]:
#在预测中为暴小冷，实际上也暴大冷或暴小冷的概率是0.8591
a.loc[(a['predict_cold']=='little_cold')&(a['cold']!='normal')].shape[0]/a.loc[(a['predict_cold']=='little_cold')].shape[0]

0.6591023701462431

In [13]:
#总的爆小冷的比例和预测的爆小冷的比例，这个预测比例太小了只有2.7%
print(a.loc[a['cold']=='little_cold'].shape[0]/a.shape[0])
print(a.loc[a['predict_cold']=='little_cold'].shape[0]/a.shape[0])

0.24921079113880426
0.02733626500875367


In [14]:
#总的爆大冷的比例，这个预测的比例太小了，只有0.2%
print(a.loc[a['cold']=='big_cold'].shape[0]/a.shape[0])
print(a.loc[a['predict_cold']=='big_cold'].shape[0]/a.shape[0])

0.22260514743386497
0.002191863911443184


In [15]:
#总的正常比例
print(a.loc[a['cold']=='normal'].shape[0]/a.shape[0])
print(a.loc[a['predict_cold']=='normal'].shape[0]/a.shape[0])

0.5281840614273308
0.9704718710798032


In [16]:
#总的预测准确率,即两列相等的准确率
a.loc[a['predict_cold']==a['cold']].shape[0]/a.shape[0]

0.528804400270192